In [3]:
# Load libraries
import os
os.sys.path.append("/home/jovyan/work/aiida-openbis/")
from aiida_openbis.utils import bisutils
import json
import numpy as np

In [4]:
# Connect to openBIS
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [5]:
# Classes
class OpenBISDatabase:
    def __init__(self):
        self.session = None
        self.schema = {
            "spaces": [], 
            "property_types": [], 
            "object_types": [],
            "experiment_types": [],
            "vocabularies": [],
            "dataSetTypes": [],
        }

    def connect_to_openbis(self, bisurl: str, bisuser: str, bispasswd: str):
        """Function to connect to openBIS"""
        self.session = bisutils.log_in(
            bisurl = bisurl, 
            bisuser = bisuser, 
            bispasswd = bispasswd
        )
    
    def extract_spaces(self):
        """Function to extract the spaces from openBIS"""
        spaces = self.session.get_spaces()

        for space in spaces:
            space_metadata = {"code": space.code, "description": space.description}
            self.schema["spaces"].append(space_metadata)

    def extract_projects(self):
        """Function to extract the projects from openBIS"""
        for index, space in enumerate(self.schema["spaces"]):
            projects = self.session.get_projects(space = space["code"])
            self.schema["spaces"][index]["projects"] = []

            for project in projects:
                project_metadata = {
                    "code": project.code
                }
                self.schema["spaces"][index]["projects"].append(project_metadata)
    
    def extract_experiments(self):
        """Function to extract the experiments from openBIS"""
        for index_space, space in enumerate(self.schema["spaces"]):
            for index_project, project in enumerate(space["projects"]):
                experiments = self.session.get_experiments(
                    space = space["code"], 
                    project = project["code"]
                )
                self.schema["spaces"][index_space]["projects"][index_project]["experiments"] = []

                for experiment in experiments:
                    experiment_metadata = {
                        "code": experiment.code, 
                        "type": experiment.type.code,
                        "properties": experiment.props.all()
                    }
                    self.schema["spaces"][index_space]["projects"][index_project]["experiments"].append(experiment_metadata)
                    
    def extract_objects(self):
        """Function to extract the objects from openBIS"""
        for index_space, space in enumerate(self.schema["spaces"]):
            for index_project, project in enumerate(space["projects"]):
                for index_experiment, experiment in enumerate(project["experiments"]):
                    objects = self.session.get_samples(
                        space = space["code"], 
                        experiment = f"/{space['code']}/{project['code']}/{experiment['code']}"
                    )
                    self.schema["spaces"][index_space]["projects"][index_project]["experiments"][index_experiment]["objects"] = []

                    for object in objects:
                        object = self.session.get_object(object.permId) #TODO: Correct this easy fix once the bug of getting parents from items from get_objects() is fixed

                        object_parents_identifiers = []
                        object_children_identifiers = []
                        object_datasets_metadata = []

                        for parent in object.get_parents():
                            object_parents_identifiers.append(parent.identifier)
                        
                        for child in object.get_children():
                            object_children_identifiers.append(child.identifier)

                        object_datasets = object.get_datasets()
                        for dataset in object_datasets:
                            dataset.data["dataStore"]["downloadUrl"] = 'https://openbis/'
                            dataset.download(destination = 'openBIS_datasets')
                            object_datasets_metadata.append({"type": dataset.type.code, "folderpath": f"openBIS_datasets/{dataset.code}"})

                        object_metadata = {
                            "code": object.code, 
                            "type": object.type.code,
                            "properties": object.props.all(),
                            "parents": object_parents_identifiers,
                            "children": object_children_identifiers,
                            "datasets": object_datasets_metadata
                        }
                        self.schema["spaces"][index_space]["projects"][index_project]["experiments"][index_experiment]["objects"].append(object_metadata)
    
    def extract_property_types(self):
        """Function to extract the property types from openBIS"""
        for property_type in self.session.get_property_types():
            property_type_metadata = {
                "code": property_type.code,
                "label": property_type.label,
                "description": property_type.description,
                "dataType": property_type.data_type,
                "vocabulary": property_type.vocabulary,
                "metaData": property_type.metaData,
            }
            self.schema["property_types"].append(property_type_metadata)
    
    def extract_object_types(self):
        """Function to extract the object types from openBIS"""
        for object_type in self.session.get_object_types():
            object_properties = object_type.get_property_assignments().df
            object_properties_metadata = []

            for _, object_property in object_properties.iterrows():
                object_properties_metadata.append(
                    {
                    "section": object_property.section,
                    "mandatory": object_property.mandatory,
                    "prop": object_property.propertyType,
                    }
                )

            object_type_metadata = {
                "code": object_type.code,
                "description": object_type.description,
                "generatedCodePrefix": object_type.generatedCodePrefix,
                "autoGeneratedCode": object_type.autoGeneratedCode,
                "propertyAssignments": object_properties_metadata
            }
            self.schema["object_types"].append(object_type_metadata)
    
    def extract_experiment_types(self):
        """Function to extract the experiment types from openBIS"""
        for experiment_type in self.session.get_experiment_types():
            experiment_properties = experiment_type.get_property_assignments().df
            experiment_properties_metadata = []

            for _, experiment_property in experiment_properties.iterrows():
                experiment_properties_metadata.append(
                    {
                    "section": experiment_property.section,
                    "mandatory": experiment_property.mandatory,
                    "prop": experiment_property.propertyType,
                    }
                )

            experiment_type_metadata = {
                "code": experiment_type.code,
                "description": experiment_type.description,
                "propertyAssignments": experiment_properties_metadata,
            }
            self.schema["experiment_types"].append(experiment_type_metadata)

    def extract_vocabularies(self):
        """Function to extract the vocabularies from openBIS"""
        for vocabulary in self.session.get_vocabularies():
            
            terms_metadata = []
            terms = vocabulary.get_terms().df
            for _, term in terms.iterrows():
                terms_metadata.append(
                    {
                    "code": term.code,
                    "label": term.label,
                    "description": term.description,
                    }
                )

            vocabulary_metadata = {
                "code": vocabulary.code,
                "description": vocabulary.description,
                "terms": terms_metadata,
            }
            self.schema["vocabularies"].append(vocabulary_metadata)
    
    def extract_dataset_types(self):
        for dataset_type in self.session.get_dataset_types():
            dataset_type_properties = dataset_type.get_property_assignments().df
            dataset_type_properties_metadata = []

            for _, dataset_type_property in dataset_type_properties.iterrows():
                dataset_type_properties_metadata.append(
                    {
                    "section": dataset_type_property.section,
                    "mandatory": dataset_type_property.mandatory,
                    "prop": dataset_type_property.propertyType,
                    }
                )

            dataset_type_metadata = {
                "code": dataset_type.code,
                "description": dataset_type.description,
                "propertyAssignments": dataset_type_properties_metadata,
            }
            self.schema["dataSetTypes"].append(dataset_type_metadata)
    
    def extract_openbis_schema(self):
        self.extract_spaces()
        self.extract_projects()
        self.extract_experiments()
        self.extract_objects()
        self.extract_property_types()
        self.extract_object_types()
        self.extract_experiment_types()
        self.extract_vocabularies()
        self.extract_dataset_types()
    
    def export_schema_to_json(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.schema, f, indent = 3)


In [6]:
# Create OpenBISDatabase instance
openbis_database = OpenBISDatabase()

# Connect to openBIS
openbis_database.connect_to_openbis("openbis", "admin", "changeit")

# Extract openBIS schema
openbis_database.extract_openbis_schema()

# Dump schema into JSON file
filename = 'openBIS_schema.json'
openbis_database.export_schema_to_json('openBIS_schema.json')

In [7]:
filename = '/home/jovyan/work/aiida-openbis/Python_Scripts/openBIS_schema.json'
with open(filename, 'r') as f:
    openbis_schema = json.load(f)

In [8]:
all_objects_codes_indexes = {}
for index_space, space in enumerate(openbis_schema["spaces"]):
    for index_project, project in enumerate(space['projects']):
        for index_experiment, experiment in enumerate(project['experiments']):
            for index_object, object in enumerate(experiment['objects']):
                all_objects_codes_indexes[object["code"]] = [index_space, index_project, index_experiment, index_object]



In [12]:
for index_space, space in enumerate(openbis_schema["spaces"]):
    new_space = session.new_space(
        code = space["code"],
        description = space["description"]
    )
    new_space.save()

{"method": "createSpaces", "params": ["admin-231220182854644xE5E157C7C511A2C72F03BA453AA66C06", [{"@type": "as.dto.space.create.SpaceCreation", "code": "DEFAULT", "description": null}]], "id": "2", "jsonrpc": "2.0"}


ValueError: Space already exists in the database and needs to be unique. (Context: [])

In [10]:
all_objects_codes.index('DEFAULT')

NameError: name 'all_objects_codes' is not defined

In [ ]:
all_objects_codes_indexes

{'DEFAULT': [0, 0, 0, 0],
 'BENCH': [1, 0, 0, 0],
 'DEFAULT_STORAGE': [1, 0, 0, 1],
 'ORDER_TEMPLATE': [1, 1, 0, 0],
 'MOL19': [5, 0, 0, 0],
 'MOL64': [5, 0, 0, 1],
 'CRYS20': [5, 1, 0, 0],
 'CRYS62': [5, 1, 0, 1],
 'ATOM_MODEL115': [5, 2, 0, 0],
 'ATOM_MODEL118': [5, 2, 0, 1],
 'PUBL68': [8, 0, 0, 0],
 'PUBL13': [8, 0, 0, 1],
 'AUTH66': [9, 0, 0, 0],
 'AUTH67': [9, 0, 0, 1],
 'AUTH65': [9, 0, 0, 2],
 'PERS1': [9, 0, 1, 0],
 'PERS2': [9, 0, 1, 1],
 'PERS3': [9, 0, 1, 2],
 'PERS4': [9, 0, 1, 3],
 'INST5': [10, 0, 0, 0],
 'INST6': [10, 0, 0, 1],
 'INST7': [10, 0, 0, 2],
 'INST8': [10, 0, 0, 3],
 'INST9': [10, 0, 0, 4],
 'MANUFPER73': [10, 1, 0, 0],
 'ROOM54': [10, 2, 0, 0],
 'ROOM55': [10, 2, 0, 1],
 'SUPPL56': [10, 3, 0, 0],
 'SUPPL57': [10, 3, 0, 1],
 'TRANSF35': [11, 0, 0, 0],
 'TRANSF63': [11, 0, 0, 1],
 'MAINT58': [11, 0, 1, 0],
 'MAINT59': [11, 0, 1, 1],
 'MAINT60': [11, 0, 1, 2],
 'MAINT61': [11, 0, 1, 3],
 'GRT52': [12, 0, 0, 0],
 'GRT53': [12, 1, 0, 0],
 'INSTR17': [13, 0, 0, 0]

In [ ]:
if ['a']:
    print('ola')

ola


In [ ]:
session.get_semantic_annotations()

,permId,entityType,propertyType,predicateOntologyId,predicateOntologyVersion,predicateAccessionId,descriptorOntologyId,descriptorOntologyVersion,descriptorAccessionId,creationDate
